# ENEM - Análise textual e extração da matriz de referência de 2010 até 2019

## Configurações do notebook

### Importação dos pacotes

In [1]:
from PyPDF2 import PdfFileReader

In [2]:
from re import sub, split, match

In [3]:
from json import dump

### Criação do dicionário contendo os caminhos dos arquivos de cada ano

In [4]:
arquivos_matriz_referencia = {
    2010: '../2010/Leia-me e Documentos Técnico/Matriz_Referencia_Enem.pdf',
    2011: '../2011/Matriz_Referencia_Enem.pdf',
    2012: '../2012/Matriz_Referencia_Enem.pdf',
    2013: '../2013/LEIA-ME e DOCUMENTOS TÉCNICOS/Matriz_Referencia_Enem.pdf',
    2014: '../2014/LEIA-ME e DOCUMENTOS TÉCNICOS/Matriz_Referencia_Enem.pdf',
    2015: '../2015/LEIA-ME e DOCUMENTOS TÉCNICOS/Matriz_Referencia_Enem.pdf',
    2016: '../2016/LEIA-ME e DOCUMENTOS TÉCNICOS/matriz_referencia_enem.pdf',
    2017: '../2017/LEIA-ME e DOCUMENTOS TÉCNICOS/Matriz_Referencia_Enem.pdf',
    2018: '../2018/LEIA-ME e DOCUMENTOS TÉCNICOS/matriz_referencia.pdf',
    2019: '../2019/LEIA-ME e DOCUMENTOS TÉCNICOS/matriz_referencia_enem.pdf',
}

## Analisando o texto dos arquivos

### Abrindo os arquivos e inseri-los em um novo dicionário

Declarando uma função para extrair os textos de cada página e juntar em uma variável única

In [5]:
def extrair_texto(arquivo_leitura_pdf):
    '''Extrai o texto de cada uma das páginas do arquivo PDF e retorna um texto único'''
    texto_arquivo = ''
    for pagina in range(arquivo_leitura_pdf.getNumPages()):
        texto_pagina = arquivo_leitura_pdf.getPage(pagina).extractText()
        texto_arquivo += texto_pagina
    return texto_arquivo

Declarando uma função para abrir os arquivos e inseri-los em um novo dicionário

In [6]:
def extrair_textos_arquivos(arquivos_matriz_referencia: dict) -> list:
    '''Abre os arquivos, inicializá-os como arquivo de leitura de PDF e extrai seu texto'''
    textos_arquivos = {}
    for ano in arquivos_matriz_referencia:
        arquivo_aberto = open(arquivos_matriz_referencia[ano], 'rb')
        arquivo_leitura_pdf = PdfFileReader(arquivo_aberto)
        texto_arquivo = extrair_texto(arquivo_leitura_pdf)
        textos_arquivos[ano] = texto_arquivo
        print(f"Texto do arquivo da matriz do ano de {ano} incluído no novo dicionário")
    return textos_arquivos

Extraindo os textos dos arquivos usando a função criada

In [7]:
textos_arquivos = extrair_textos_arquivos(arquivos_matriz_referencia)

Texto do arquivo da matriz do ano de 2010 incluído no novo dicionário
Texto do arquivo da matriz do ano de 2011 incluído no novo dicionário
Texto do arquivo da matriz do ano de 2012 incluído no novo dicionário
Texto do arquivo da matriz do ano de 2013 incluído no novo dicionário
Texto do arquivo da matriz do ano de 2014 incluído no novo dicionário
Texto do arquivo da matriz do ano de 2015 incluído no novo dicionário
Texto do arquivo da matriz do ano de 2016 incluído no novo dicionário
Texto do arquivo da matriz do ano de 2017 incluído no novo dicionário
Texto do arquivo da matriz do ano de 2018 incluído no novo dicionário
Texto do arquivo da matriz do ano de 2019 incluído no novo dicionário


Declarando uma função para verificar se os textos são iguais

In [8]:
def verificar_igualdade_textos(textos_arquivos: dict) -> None:
    '''Verifica se os textos dos arquivos das matrizes de referência são iguais'''
    anos = [ano for ano in textos_arquivos]
    for indice, ano in enumerate(anos[:-1]):
        if textos_arquivos[ano] == textos_arquivos[anos[indice + 1]]:
            resultado = 'iguais'
        else:
            resultado = 'diferentes'
        print(f"Texto dos anos de {ano} e {ano + 1} são {resultado}")

Verificando a igualdade dos textos usando a função criada

In [9]:
verificar_igualdade_textos(textos_arquivos)

Texto dos anos de 2010 e 2011 são iguais
Texto dos anos de 2011 e 2012 são iguais
Texto dos anos de 2012 e 2013 são iguais
Texto dos anos de 2013 e 2014 são iguais
Texto dos anos de 2014 e 2015 são iguais
Texto dos anos de 2015 e 2016 são iguais
Texto dos anos de 2016 e 2017 são iguais
Texto dos anos de 2017 e 2018 são iguais
Texto dos anos de 2018 e 2019 são iguais


## Extração das habilidades da matriz de referência do ano de 2019

Uma vez que todos os arquivos das matrizes dos anos de 2010 até 2019 possuem o mesmo texto, podemos extrair as habilidades apenas do arquivo do ano de 2019.

Criando um objeto com textos dos títulos de cada uma das provas para utilizá-los como ponto de corte das competências e das habilidades

In [10]:
titulos_provas = {
    'LC': 'Matriz de Referência de Linguagens, Códigos e suas Tecnologias',
    'MT': 'Matriz de Referência de Matemática e suas Tecnologias',
    'CN': 'Matriz de Referência de Ciências da Natureza e suas Tecnologias',
    'CH': 'Matriz de Referência de Ciências Humanas e suas Tecnologias'
}

### Extraindo o texto das matrizes de referência das provas do ano de 2019

Excluindo o caractere de quebra de linha ("\n")

In [11]:
texto_matriz_2019 = textos_arquivos[2019].replace('\n', '')

Substituindo o espaço duplo por simples

In [12]:
texto_matriz_2019 = texto_matriz_2019.replace('  ', ' ')

Declarando uma função para separar o texto de cada prova e salvá-lo em um novo dicionário

In [13]:
def separar_texto_prova(texto_matriz: str, titulos_provas: dict) -> dict:
    '''Separa os textos das provas usando os títulos para definir os pontos de corte'''
    siglas_provas = [sigla_prova for sigla_prova in titulos_provas]
    textos_provas = {}
    for indice, sigla_prova in enumerate(siglas_provas):
        titulo_prova = titulos_provas[sigla_prova]
        if indice + 1 < len(titulos_provas):
            texto_limite = titulos_provas[siglas_provas[indice + 1]]
        else:
            texto_limite = 'ANEXO'
        texto_prova = texto_matriz.partition(titulos_provas[sigla_prova])[2].partition(texto_limite)[0]
        textos_provas[sigla_prova] = titulo_prova + texto_prova
        print(f"Texto da prova de {sigla_prova} extraído com sucesso")
    return textos_provas 

Separando os textos usando a função criada

In [14]:
textos_provas = separar_texto_prova(texto_matriz_2019, titulos_provas)

Texto da prova de LC extraído com sucesso
Texto da prova de MT extraído com sucesso
Texto da prova de CN extraído com sucesso
Texto da prova de CH extraído com sucesso


Mostrando o texto separado da prova de Matemática e suas Tecnologias

In [15]:
textos_provas['MT']

'Matriz de Referência de Matemática e suas Tecnologias Competência de área 1 - Construir significados para os números naturais, inteiros, racionais e reais. H1 - Reconhecer, no contexto social, diferentes significados e representações dos números e operações - naturais, inteiros, racionais ou reais. H2 - Identificar padrões numéricos ou princípios de contagem. H3 - Resolver situação-problema envolvendo conhecimentos numéricos. H4 - Avaliar a razoabilidade de um resultado numérico na construção de argumentos sobre afirmações quantitativas. H5 - Avaliar propostas de intervenção na realidade utilizando conhecimentos numéricos. Competência de área 2 - Utilizar o conhecimento geométrico para realizar a leitura e a representação da realidade e agir sobre ela. H6 - Interpretar a localização e a movimentação de pessoas/objetos no espaço tridimensional e sua representação no espaço bidimensional. H7 - Identificar características de figuras planas ou espaciais. H8 - Resolver situação-problema qu

### Construindo os objetos com as matrizes de cada prova

Declarando uma função para construir o dicionário da matriz das provas

In [16]:
def construir_matriz_prova(texto: str) -> dict:
    '''Constrói o dicionário contendo os dados da matriz de referência de uma prova'''
    texto_com_quebra = sub(r"(Competência de área \d{1,2} ?-? )|(H\d{1,2})", r"\n\g<0>", texto)
    lista_itens = split(r" \n", texto_com_quebra)
    lista_competencias = []
    lista_habilidades = []
    for item in lista_itens[1:]:
        if item.startswith('Competência'):
            grupos = match(r"Competência de área (\d{1,2}) ?-? (.*)\.?", item).groups()
            nova_competencia = {
                'id_competencia': grupos[0],
                'texto_competencia': grupos[1].strip(),
            }
            lista_competencias.append(nova_competencia)
        elif item.startswith('H'):
            grupos = match(r"H(\d{1,2}) ?-? (.*)\.?", item).groups()
            nova_habilidade = {
                'id_habilidade': grupos[0],
                'texto_habilidade': grupos[1].strip(),
                'id_competencia': lista_competencias[-1]['id_competencia'],
            }
            lista_habilidades.append(nova_habilidade)
    matriz = {
        'titulo': lista_itens[0],
        'competencias': lista_competencias,
        'habilidades': lista_habilidades,
    }
    return matriz

Declarando uma função para construir todas as matrizes das provas e armazenar num novo dicionário

In [17]:
def construir_todas_matrizes(textos_provas: dict) -> dict:
    '''Constrói o dicionário contendo todos os dicionários com os dados das matrizes de referência das provas'''
    dicionario_matrizes = {}
    for sigla_prova in textos_provas:
        matriz_prova = construir_matriz_prova(textos_provas[sigla_prova])
        dicionario_matrizes[sigla_prova] = matriz_prova
        print(f"Matriz da prova {sigla_prova} construído com sucesso")
    return dicionario_matrizes        

Construindo todas as matrizes usando a função criada

In [18]:
matrizes = construir_todas_matrizes(textos_provas)

Matriz da prova LC construído com sucesso
Matriz da prova MT construído com sucesso
Matriz da prova CN construído com sucesso
Matriz da prova CH construído com sucesso


Mostrando a matriz da prova de Matemática e suas Tecnologias

In [19]:
matrizes['MT']

{'titulo': 'Matriz de Referência de Matemática e suas Tecnologias',
 'competencias': [{'id_competencia': '1',
   'texto_competencia': 'Construir significados para os números naturais, inteiros, racionais e reais.'},
  {'id_competencia': '2',
   'texto_competencia': 'Utilizar o conhecimento geométrico para realizar a leitura e a representação da realidade e agir sobre ela.'},
  {'id_competencia': '3',
   'texto_competencia': 'Construir noções de grandezas e medidas para a compreensão da realidade e a solução de problemas do cotidiano.'},
  {'id_competencia': '4',
   'texto_competencia': 'Construir noções de variação de grandezas para a compreensão da realidade e a solução de problemas do cotidiano.'},
  {'id_competencia': '5',
   'texto_competencia': 'Modelar e resolver problemas que envolvem variáveis socioeconômicas ou técnico-científicas, usando representações algébricas.'},
  {'id_competencia': '6',
   'texto_competencia': 'Interpretar informações de natureza científica e social obt

Declarando uma função para imprimir apenas as competências da matriz

In [20]:
def imprimir_competencias(matriz: dict) -> None:
    '''Imprime as competências da matriz de referência da prova passada como argumento'''
    for competencia in matriz['competencias']:
        print(f"Competência de área {competencia['id_competencia']} - {competencia['texto_competencia']}")

Imprimindo as competências da prova de Matemática e suas Tecnologias usando a função criada

In [21]:
imprimir_competencias(matrizes['MT'])

Competência de área 1 - Construir significados para os números naturais, inteiros, racionais e reais.
Competência de área 2 - Utilizar o conhecimento geométrico para realizar a leitura e a representação da realidade e agir sobre ela.
Competência de área 3 - Construir noções de grandezas e medidas para a compreensão da realidade e a solução de problemas do cotidiano.
Competência de área 4 - Construir noções de variação de grandezas para a compreensão da realidade e a solução de problemas do cotidiano.
Competência de área 5 - Modelar e resolver problemas que envolvem variáveis socioeconômicas ou técnico-científicas, usando representações algébricas.
Competência de área 6 - Interpretar informações de natureza científica e social obtidas da leitura de gráficos e tabelas, realizando previsão de tendência, extrapolação, interpolação e interpretação.
Competência de área 7 - Compreender o caráter aleatório e não-determinístico dos fenômenos naturais e sociais e utilizar instrumentos adequados 

Declarando uma função para imprimir apenas as competências da matriz

In [22]:
def imprimir_habilidades(matriz: dict) -> None:
    '''Imprime as habilidades da matriz de referência da prova'''
    for habilidade in matriz['habilidades']:
        print(f"H{habilidade['id_habilidade']} - {habilidade['texto_habilidade']}")

Imprimindo as habilidades da prova de Matemática e suas Tecnologias usando a função criada

In [23]:
imprimir_habilidades(matrizes['MT'])

H1 - Reconhecer, no contexto social, diferentes significados e representações dos números e operações - naturais, inteiros, racionais ou reais.
H2 - Identificar padrões numéricos ou princípios de contagem.
H3 - Resolver situação-problema envolvendo conhecimentos numéricos.
H4 - Avaliar a razoabilidade de um resultado numérico na construção de argumentos sobre afirmações quantitativas.
H5 - Avaliar propostas de intervenção na realidade utilizando conhecimentos numéricos.
H6 - Interpretar a localização e a movimentação de pessoas/objetos no espaço tridimensional e sua representação no espaço bidimensional.
H7 - Identificar características de figuras planas ou espaciais.
H8 - Resolver situação-problema que envolva conhecimentos geométricos de espaço e forma.
H9 - Utilizar conhecimentos geométricos de espaço e forma na seleção de argumentos propostos como solução de problemas do cotidiano.
H10 - Identificar relações entre grandezas e unidades de medida.
H11 - Utilizar a noção de escalas na

Declarando uma função para imprimir a matriz completa

In [24]:
def imprimir_matriz_completa(matriz: dict) -> None:
    '''Imprime a matriz de referência completa da prova'''
    print(matriz['titulo'])
    for competencia in matriz['competencias']:
        print(f"Competência de área {competencia['id_competencia']} - {competencia['texto_competencia']}")
        lista_habilidades = [habilidade for habilidade in matriz['habilidades'] if habilidade['id_competencia'] == competencia['id_competencia']]
        for habilidade in lista_habilidades:
            print(f"H{habilidade['id_habilidade']} - {habilidade['texto_habilidade']}")

Imprimindo a matriz completa da prova de Matemática e suas Tecnologias usando a função criada

In [25]:
imprimir_matriz_completa(matrizes['MT'])

Matriz de Referência de Matemática e suas Tecnologias
Competência de área 1 - Construir significados para os números naturais, inteiros, racionais e reais.
H1 - Reconhecer, no contexto social, diferentes significados e representações dos números e operações - naturais, inteiros, racionais ou reais.
H2 - Identificar padrões numéricos ou princípios de contagem.
H3 - Resolver situação-problema envolvendo conhecimentos numéricos.
H4 - Avaliar a razoabilidade de um resultado numérico na construção de argumentos sobre afirmações quantitativas.
H5 - Avaliar propostas de intervenção na realidade utilizando conhecimentos numéricos.
Competência de área 2 - Utilizar o conhecimento geométrico para realizar a leitura e a representação da realidade e agir sobre ela.
H6 - Interpretar a localização e a movimentação de pessoas/objetos no espaço tridimensional e sua representação no espaço bidimensional.
H7 - Identificar características de figuras planas ou espaciais.
H8 - Resolver situação-problema que

## Salvando o objeto para usar em trabalhos futuros

Inicializando o arquivo para salvar os dados

In [26]:
out_file = open("matrizes.json", "w")

Salvando os dados no arquivo

In [27]:
dump(matrizes, out_file, indent=2)

Fechando o arquivo

In [28]:
out_file.close()